In [2]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

from utils import get_face_rect, calculate_adaptive_skin_mask, calculate_mean_deviation

In [4]:
# head tracking with camshift
cap = cv.VideoCapture(0)

hand_cascade = cv.CascadeClassifier('palm.xml')

while True:
    _, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    palm = hand_cascade.detectMultiScale(gray)
    
    if len(palm) == 0:
        continue
    else:
        for (x, y, w, h) in palm:
            cv.rectangle(rgb, (x, y), (x+w, y+h), (255, 0, 0), 1)
            palm_rect = (x, y, w, h)
            palm_roi = rgb[y:y+h, x:x+w]
        break

_, face_rect = get_face_rect(cap)
vals = calculate_mean_deviation(cap)

track_window = (0, 0, 250, 250)
roi = frame[0:250, 0:250]

rgb_roi = cv.cvtColor(roi, cv.COLOR_BGR2RGB)
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = calculate_adaptive_skin_mask(rgb_roi, vals)

roi_hist = cv.calcHist([hsv_roi], [0], mask, [0], [0, 180])
cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)

term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)

while True:
    _, frame = cap.read()
    rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    mask = calculate_adaptive_skin_mask(rgb, vals)
    dst = cv.calcBackProject([rgb], [0], roi_hist, [0, 256], 1)

    
    _, track_window = cv.CamShift(dst, track_window, term_crit)

    x, y, w, h = track_window
    cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0))

    x1, y1, w1, h1 = palm_rect
    cv.rectangle(frame, (x1-25, y1-25), (x1+w1+25, y1+h1+25), (0, 0, 255))


    cv.imshow('dst', dst)
    cv.imshow('frame', frame)

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

error: OpenCV(4.5.2) /tmp/pip-req-build-rcdoave0/opencv/modules/imgproc/src/histogram.cpp:993: error: (-215:Assertion failed) mask.empty() || mask.type() == CV_8UC1 in function 'calcHist'


In [5]:
cap = cv.VideoCapture(0)

ret, frame = cap.read()

print(frame.shape)#640x360
x, y, w, h = 100, 100, 300, 300
track_window = (x, y, w, h)

roi = frame[y:y + h, x:x + w]
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = cv.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv.calcHist([hsv_roi], [0], mask, [180], [0, 180])#Calculate histogram
cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)

term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = cap.read()
    if ret is True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
       
        ret, track_window = cv.meanShift(dst, track_window, term_crit)
        
        x, y, w, h = track_window
        
        img2 = cv.rectangle(frame, (x, y), (x + w, y + h), (255,0,0), 2)
        cv.imshow('img2', img2)
        cv.imshow('dst', dst)
       
        if cv.waitKey(1) == ord('q'):
            break
    else:
        break
cv.destroyAllWindows()
cap.release()

(480, 640, 3)


In [5]:
cap.release()

In [17]:
cap = cv.VideoCapture(0)
i = 0
j = 0

ret,frame = cap.read()

x, y, w, h = 0, 0, 250, 250 # simply hardcoded the values
track_window = (x, y, w, h)

roi = frame[y:y+h, x:x+w]
hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)
_, face_rect = get_face_rect(cap)
vals = calculate_mean_deviation(cap)
mask = calculate_adaptive_skin_mask(roi, vals, hide_rect=face_rect)

mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)

term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )
while(1):
    ret, frame = cap.read()
    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)
       
        ret, track_window = cv.meanShift(dst, track_window, term_crit)
        
        x,y,w,h = track_window
        img2 = cv.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        
        cv.imshow('img2',img2)
        cv.imshow('dst', dst)
        
        if i == 60:
            i = 0
            cv.imwrite(f'../tracking/{j}.jpg', img2)
            j += 1
            print('Saved image')
        else:
            i += 1

        if cv.waitKey(1) == ord('q'):
            break
    else:
        break

cap.release()
cv.destroyAllWindows()

Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image
Saved image


In [16]:
cap.release()